# Hot Sauce NLP


### web scraping

In [77]:
import requests
from bs4 import BeautifulSoup

import numpy as np
import pandas as pd
                    
%matplotlib inline
import matplotlib
import random

In [2]:
def fetch_page(page):
    print(f"scraping page {page + 1}")
    response = requests.get(
        f'https://hotsaucefever.com/hotsauce/',
        headers={"Accept-Language":"en-US"}
    )
    soup = BeautifulSoup(response.content, "html.parser")
    return soup

In [3]:

def add_sauces_to_dict(soup, dict):
    for sauce_html in soup.find_all(class_='detail'):
        dict['Hot Sauce'].append(sauce_html.find('h2').find('a').string)
        dict['Rating'].append(sauce_html.find('h3').find('span').string)
        dict['Makers'].append(
            sauce_html.find(class_="list-txt").get_text())

In [4]:
def create_sauces_df(max_page):
    sauces_dict = {'Hot Sauce': [], 'Rating': [], 'Makers': []}
    for page in range(max_page):
        soup = fetch_page(page)
        add_sauces_to_dict(soup, sauces_dict)
    return pd.DataFrame.from_dict(sauces_dict)

In [5]:
sauces_df = create_sauces_df(25)
sauces_df.shape

scraping page 1
scraping page 2
scraping page 3
scraping page 4
scraping page 5
scraping page 6
scraping page 7
scraping page 8
scraping page 9
scraping page 10
scraping page 11
scraping page 12
scraping page 13
scraping page 14
scraping page 15
scraping page 16
scraping page 17
scraping page 18
scraping page 19
scraping page 20
scraping page 21
scraping page 22
scraping page 23
scraping page 24
scraping page 25


(625, 3)

### Sorting the DF

In [6]:
# new data frame with split value columns
new = sauces_df["Makers"].str.split("Maker:", n =1, expand = True)


In [7]:
new1 = new[1].str.split(":", expand = True)


#### stripping unwanted html

In [8]:
cols = [0, 1, 2]
new1[cols] = new1[cols].apply(lambda x: x.str.strip('%\n'))
new1[cols] = new1[cols].apply(lambda x: x.str.strip(' \n%'))
new1[cols] = new1[cols].apply(lambda x: x.str.strip('%\n'))
new1[cols] = new1[cols].apply(lambda x: x.str.strip('%\nPepper(s)'))
new1[cols] = new1[cols].apply(lambda x: x.str.strip('\n'))
new1[cols] = new1[cols].apply(lambda x: x.str.strip('SHU'))

#### Merging 2 DF's

In [9]:
result = pd.concat([sauces_df, new1], axis=1, ignore_index=True)
result.drop(result.columns[[2]], axis=1, inplace=True)
new2 = result[0].str.split(" - ", expand = True)

In [53]:
result1 = pd.concat([result, new2], axis=1, ignore_index=True)
result1.drop(result1.columns[[0]], axis=1, inplace=True)
result1.columns = ['Rating', 'Address', 'Peppers', 'Scoville', 'Company', 'Sauce']
result1 = result1[['Sauce', 'Company', 'Rating','Peppers', 'Scoville', 'Address']]

In [54]:
result1.drop(result1.columns[[4]], axis=1, inplace=True)

In [55]:
result1['Rating'] = result1['Rating'].replace(np.nan, 0)

#### Randomise Rating Due to lack of info

In [96]:
np.random.seed(365)
result1['Rating'] = np.round(np.random.uniform(3.2, 5.0, size=(len(result1), 1)), 1)

### Data Visualisation

In [99]:
result1

,Sauce,Company,Rating,Peppers,Address
0,Jalapeño,Whitehouse Station Sauce Company,4.9,Jalapeño,"Whitehouse Station Sauce Company, Whitehouse S..."
1,Black Garlic,Jersey Barnfire,4.4,"abanero, Jalapeño","Jersey Barnfire, Sparta, NJ, United States\n ..."
2,Jah Mama Sauce,Roofeeo Landis,4.4,"cotch Bonnet, Fresno","Roofeeo Landis, Los Angeles , CA, United State..."
3,Purple Haze Double Dose,Ric Orlando's Best,4.3,"abanero, Carolina Rea","Ric Orlando's Best, West Hurley, NY, United St..."
4,Pepper Pastry,Hell's Kitchen Hot Sauce,4.2,"abanero, Fatalii","ell's Kitchen Hot Sauce, New York, NY, United ..."
...,...,...,...,...,...
620,Fermented Habanero Hot Sauce,Iggy's,3.5,abanero,"Iggy's Alive & Cultured, Bainbridge Island, WA..."
621,Kelp Hot Sauce,Bullwhip,4.9,i Peri,"Barnacle Foods, Juneau, AK, United States\n ..."
622,Moroccan Chile Crisp,oo'mämē,4.3,None,"Fifth Taste Foods, LLC, St Louis, MO, United Stat"
623,Indian Chile Crisp,oo'mämē,4.6,None,"Fifth Taste Foods, LLC, St Louis, MO, United Stat"
